# **Projek Akhir Praktikum Data Science**

> Analisis Sentimen Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier dan NLP Pada Sosial Media Twitter

**Oleh Kelompok 2 :**
1. Hazlan Muhammad Qodri (123190080)
2. Elisia Dwi Rahayu (123190062)
3. Shania Septika Inayasari (123190055)

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

# **1. Scraping Data from Twitter**

In [ ]:
%pip install tweepy

In [10]:
import tweepy
import csv

#### Secret Env Variable

In [17]:
import os

consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_key_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')
bearer_token = os.environ.get('bearer_token')

In [18]:
# token for twitter developers
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret,
)

response = client.get_home_timeline()
print(response)


Response(data=[<Tweet id=1592726621819060224 text='[cm] kemaren aku abis cek plagiarisme buat proposal UP (di cek sama dosennya). kalo hasilnya nol gini beneran ga kena plagiarisme? atau ada kesalahan ya? aku kaget banget kok bisa nol\U0001f972 kan harusnya engga yaa? https://t.co/xIa7ZTP0H4'>, <Tweet id=1592726617498927104 text='[cm] cari info beasiswa dmn ya biasanya'>, <Tweet id=1592726590001029121 text='/LT Drop buku self development andalan kalian yuk! Sender mau baca buku sekalian belajar lagi. Thank you 🌼'>, <Tweet id=1592726581084323842 text='gimana caranya bertahan ditengah badai deadline, revisi dan tekanan dari pembimbing tanpa support sistem. bagi tips please, dah mleyot bgt 😭😭 [cm]'>, <Tweet id=1592726506610229253 text='[cm] temenku mau sempro, perlu ngasih sesuatu ga sih? ngasih apa ya? temenku cuman dia doang selama kuliah jd gatau kalo temen sempro biasanya harus ngapain☹️☹️'>, <Tweet id=1592726420174024705 text='RT @noya_steffany: Good morning, tweeps!\n\nAn IT consult